# Depth First Search

54 - C  
https://atcoder.jp/contests/abc054/tasks/abc054_c

In [1]:
N, M = 3, 4
edges = [[1, 2], [1, 3], [2, 3]]

In [2]:
graph = [[None] * N for _ in range(N)]
graph

[[None, None, None], [None, None, None], [None, None, None]]

In [3]:
for x, y in edges:
    graph[x - 1][y - 1] = True
    graph[y - 1][x - 1] = True

In [4]:
def dfs(v):    
    if all(visited):
        return 1
    
    ret = 0
    for i in range(N):
        if not graph[v][i]:
            continue
        if visited[i]:
            continue
            
        visited[i] = True
        ret += dfs(i)
        visited[i] = False
        
    return ret

In [5]:
visited = [True] + [False] * (N - 1)
print(dfs(0))

2


# Warshall-Floyd's algorithm

全点対間　最短経路問題　(All Pairs Shortest Path)

計算量は$O(N^3)$

例） 22 - C  
https://atcoder.jp/contests/abc022/tasks/abc022_c

例） 16 - C  
https://atcoder.jp/contests/abc016/tasks/abc016_3

In [22]:
N, M = 5, 7
edges = [[1, 2, 2], [1, 4, 1], [2, 3, 7], [1, 5, 12],
         [3, 5, 2], [2, 5, 3], [3, 4, 5]]

In [34]:
graph = [[float('inf')] * N for _ in range(N)]
for i in range(N):
    graph[i][i] = 0

for u, v, l in edges:
    graph[u - 1][v - 1] = l
    graph[v - 1][u - 1] = l
    
graph

[[0, 2, inf, 1, 12],
 [2, 0, 7, inf, 3],
 [inf, 7, 0, 5, 2],
 [1, inf, 5, 0, inf],
 [12, 3, 2, inf, 0]]

In [35]:
for k in range(N):
    for i in range(N):
        if graph[i][k] == float('inf'):
            continue
        for j in range(N):
            if graph[k][j] == float('inf'):
                continue
            graph[i][j] = min(graph[i][j], graph[i][k] + graph[k][j])
            
graph

[[0, 2, 6, 1, 5],
 [2, 0, 5, 3, 3],
 [6, 5, 0, 5, 2],
 [1, 3, 5, 0, 6],
 [5, 3, 2, 6, 0]]

#### もしくは，scipyを使う

https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csgraph.floyd_warshall.html

In [34]:
graph = [[float('inf')] * N for _ in range(N)]
for i in range(N):
    graph[i][i] = 0

for u, v, l in edges:
    graph[u - 1][v - 1] = l
    graph[v - 1][u - 1] = l
    
graph

[[0, 2, inf, 1, 12],
 [2, 0, 7, inf, 3],
 [inf, 7, 0, 5, 2],
 [1, inf, 5, 0, inf],
 [12, 3, 2, inf, 0]]

In [33]:
from scipy.sparse.csgraph import floyd_warshall

graph = floyd_warshall(graph, directed=True)
graph

array([[0., 2., 6., 1., 5.],
       [2., 0., 5., 3., 3.],
       [6., 5., 0., 5., 2.],
       [1., 3., 5., 0., 6.],
       [5., 3., 2., 6., 0.]])

# Dijkstra's algorithm

始点固定の最短経路

https://atcoder.jp/contests/abc021/tasks/abc021_c

In [42]:
N = 7
edges = [[1, 2], [1, 3], [4, 2], [4, 3], [4, 5],
         [4, 6], [7, 5], [7, 6]]

### 隣接行列を使った方法 $O(V^2)$

In [69]:
graph = [[float('inf')] * N for _ in range(N)]
for i in range(N):
    graph[i][i] = 0

for u, v in edges:
    graph[u - 1][v - 1] = 1
    graph[v - 1][u - 1] = 1

graph

[[0, 1, 1, inf, inf, inf, inf],
 [1, 0, inf, 1, inf, inf, inf],
 [1, inf, 0, 1, inf, inf, inf],
 [inf, 1, 1, 0, 1, 1, inf],
 [inf, inf, inf, 1, 0, inf, 1],
 [inf, inf, inf, 1, inf, 0, 1],
 [inf, inf, inf, inf, 1, 1, 0]]

d: distance  
p: parent

In [70]:
INF = float('inf')
visited = [False] * N
d = [INF] * N
p = [INF] * N

In [71]:
init = 0
d[init] = 0
p[init] = -1

while True:
    min_cost = INF
    for i in range(N):
        if not visited[i] and d[i] < min_cost:
            min_cost = d[i]
            u = i
            
    if min_cost == INF:
        break

    visited[u] = True
    
    for v in range(N):
        if not visited[v] and graph[u][v] < INF:
            if d[u] + graph[u][v] < d[v]:
                d[v] = d[u] + graph[u][v]
                p[v] = u

始点からの，各ノードまでの最短経路

In [72]:
d

[0, 1, 1, 2, 3, 3, 4]

最短経路の場合の親．

In [73]:
p

[-1, 0, 0, 1, 3, 3, 4]

## Priority Queueを使った実装 $O(E \log V)$

https://docs.python.org/ja/3/library/heapq.html

In [1]:
N = 7
edges = [[1, 2], [1, 3], [4, 2], [4, 3], [4, 5],
         [4, 6], [7, 5], [7, 6]]

heapqを使って，コストの低い順に並べている

In [6]:
from heapq import heappop, heappush
INF = float('inf')

d = [INF] * N
d[0] = 0

visited = [False] * N
visited[0] = True

# Priority queue (cost, node)
pq = []
heappush(pq, (0, 0))

while len(pq) > 0:
    cost, u = heappop(pq)
    visited[u] = True
    
    # Search nodes connected with target node 'u'
    for v, w in edges:
        if v - 1 != u and w - 1 != u:
            continue
            
        node = w - 1 if v - 1 == u else v - 1
        if visited[node]:
            continue
            
        # Edge cost == 1
        if d[node] > d[u] + 1:
            d[node] = d[u] + 1
            heappush(pq, (d[node], node))
            
d

[0, 1, 1, 2, 3, 3, 4]

# Topological Sort

DAGの並び替え

In [6]:
N, M = 6, 6
edges = [[0, 1], [1, 2], [3, 1], [3, 4], [4, 5], [5, 2]]

print('ans: 0 3 1 4 5 2')

ans: 0 3 1 4 5 2


## Breath First Search

In [7]:
visited = [False] * N

indeg = [0] * N
for _, j in edges:
    indeg[j] += 1
print(indeg)

[0, 2, 2, 0, 1, 1]


In [8]:
from collections import deque

def bfs(s):
    q = deque()
    q.append(s)
    visited[s] = True
    
    while len(q) > 0:
        u = q.popleft()
        ans.append(u)
        
        for c, v in edges:
            if c != u:
                continue
            
            indeg[v] -= 1
            if indeg[v] == 0 and not visited[v]:
                visited[v] = True
                q.append(v)

In [9]:
ans = []
for i in range(N):
    if indeg[i] == 0 and not visited[i]:
        bfs(i)
        
ans

[0, 3, 1, 4, 5, 2]

## Depth First Search

In [14]:
def dfs(u):
    visited[u] = True
    for c, v in edges:
        if c != u:
            continue

        if not visited[v]:
            dfs(v)
            
    ans.append(u)

In [15]:
visited = [False] * N
ans = []

for i in range(N):
    if not visited[i]:
        dfs(i)
        
ans[::-1]

[3, 4, 5, 0, 1, 2]

#### 最短経路のDAGを作る

In [18]:
from itertools import product

N = 7
d = [0, 1, 1, 2, 3, 3, 4]

dag_edges = []
for i, j in product(range(N), repeat=2):
    # Cost == 1
    if d[i] + 1 == d[j]:
        dag_edges.append([i, j])
        
dag_edges

[[0, 1], [0, 2], [1, 3], [2, 3], [3, 4], [3, 5], [4, 6], [5, 6]]

# 二部グラフ判定

隣接するノードを二色で塗り分けられるグラフを，二部グラフと呼ぶ．

In [1]:
N, M = 6, 5
edges = [[1, 2], [2, 3], [3, 4], [4, 5], [5, 6]]

print('ans: 4')

ans: 4


In [2]:
graph = [[] for _ in range(N)]
for x, y in edges:
    graph[x - 1].append(y - 1)
    graph[y - 1].append(x - 1)

In [3]:
def dfs(v, c):
    # c: color = 1 or -1
    node[v] = c
    for i in graph[v]:
        if node[i] == c:
            return False
        
        if node[i] == 0 and not dfs(i, -c):
            return False
        
    return True

グラフが連結でない場合は，各頂点から始める．  
連結の場合は，0スタートだけで良い．

In [4]:
import sys
sys.setrecursionlimit(100000)

node = [0] * N
for i in range(N):
    if node[i] == 0 and not dfs(i, 1):
        print('No')
        break
else:
    print('Yes')

Yes


#### dequeを使う

In [35]:
from collections import deque

def dfs(s, c):
    q = deque()
    q.append(s)
    visited[s] = True
    color[s] = c
    
    while len(q) > 0:
        v = q.pop()
        for i in graph[v]:
            if visited[i] and color[i] == color[v]:
                return False
            
            if not visited[i]:
                visited[i] = True
                color[i] = -color[v]
                q.append(i)
                
    return True

In [36]:
visited = [False] * N
color = [0] * N

if dfs(0, 1):
    x = sum(v + 1 for v in color) // 2
    print(x * (N - x) - M)
else:
    print(N * (N - 1) // 2 - M)

4
